In [1]:
import datetime
from glob import glob
import math
import matplotlib.pyplot as plt
import nibabel as nb
from nilearn import image
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import timeit
import warnings
%matplotlib inline

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import os
import warnings

import tensorflow as tf
import src.models.basic_cnn as model
from src.data_io import InputFnFactory, _get_data
import datetime

tf.logging.set_verbosity(tf.logging.INFO)

if __name__ == '__main__':
    log_dir = "logs"
    current_run_subdir = os.path.join(
        "run_" + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
#    model_dir = os.path.join(log_dir, model.name, "106x128x110")#current_run_subdir)
    model_dir = os.path.join(log_dir, model.name, '5272018')

    run_config = tf.estimator.RunConfig(model_dir=model_dir)

    params = tf.contrib.training.HParams(
        target_shape=(106, 128, 110),
        model_dir=model_dir
    )

    ds = InputFnFactory(target_shape=params.target_shape,
                        n_epochs=100,
                        train_src_folder="/home/smantra/finalproject/data/",
                        train_cache_prefix="/home/smantra/finalproject/cache_train_hires/",
                        eval_src_folder="/home/smantra/finalproject/eval/",
                        eval_cache_prefix="/home/smantra/finalproject/cache_eval_hires/",
                        batch_size=4
                        )

    # Workaround for cache iterator concurency issues. Iterate over the whole
    # training dataset without counterbalancing to make sure everything is
    # preprocessed and cached
    if not os.path.exists(ds.train_cache_prefix + ".index"):
        print("Preprocessing the training set")
        with tf.Session() as sess:
            train_dataset = _get_data(batch_size=ds.batch_size,
                                      src_folder=ds.train_src_folder,
                                      n_epochs=1,
                                      cache_prefix=ds.train_cache_prefix,
                                      shuffle=False,
                                      target_shape=params.target_shape,
                                      balance_dataset=False)

            train_dataset = train_dataset.make_one_shot_iterator()
            while True:
                try:
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore")
                        features, labels = sess.run(train_dataset.get_next())
                except tf.errors.OutOfRangeError:
                    break
        print("Finished preprocessing the training set")

    train_spec = tf.estimator.TrainSpec(input_fn=ds.train_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=ds.eval_input_fn,
                                      steps=None,
                                      start_delay_secs=0,
                                      throttle_secs=1200)

    estimator = tf.estimator.Estimator(model_fn=model.model_fn,
                                       params=params,
                                       config=run_config)

    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': 'logs/basic_cnn_v15/5272018', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7cd83a0278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 1200 secs (eval_spec.throttle_secs) or training is finished.
<TensorSliceDataset shapes: ((),), types: (tf.string,)>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Ch

ResourceExhaustedError: logs/basic_cnn_v15/5272018/model.ckpt-19_temp_ab1c974932e54fdebb51aad82c5dd2a4/part-00000-of-00001.data-00000-of-00001.tempstate2141628906297656134; No space left on device
	 [[Node: save/SaveV2 = SaveV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](save/ShardedFilename, save/SaveV2/tensor_names, save/SaveV2/shape_and_slices, beta1_power, beta2_power, conv3d/bias, conv3d/bias/Adam, conv3d/bias/Adam_1, conv3d/kernel, conv3d/kernel/Adam, conv3d/kernel/Adam_1, conv3d_1/bias, conv3d_1/bias/Adam, conv3d_1/bias/Adam_1, conv3d_1/kernel, conv3d_1/kernel/Adam, conv3d_1/kernel/Adam_1, dense/bias, dense/bias/Adam, dense/bias/Adam_1, dense/kernel, dense/kernel/Adam, dense/kernel/Adam_1, dense_1/bias, dense_1/bias/Adam, dense_1/bias/Adam_1, dense_1/kernel, dense_1/kernel/Adam, dense_1/kernel/Adam_1, global_step)]]
Hint: If you want to see a list of allo

In [ ]:
aws s3 ls s3://fcp-indi/data/Projects/ABIDE/RawDataBIDS/ --recursive --include "*T1w*"
aws s3 cp s3://fcp-indi/data/Projects/ABIDE/RawDataBIDS/ . --recursive --exclude "*" --include "*T1w*"

aws s3 ls s3://fcp-indi/data/Projects/ABIDE/
aws s3 ls s3://fcp-indi/data/Projects/ABIDE/RawDataBIDS/
    